# Create Dataframe

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

flights_data = [(1,'Flight1' , 'Delhi' , 'Hyderabad'),
 (1,'Flight2' , 'Hyderabad' , 'Kochi'),
 (1,'Flight3' , 'Kochi' , 'Mangalore'),
 (2,'Flight1' , 'Mumbai' , 'Ayodhya'),
 (2,'Flight2' , 'Ayodhya' , 'Gorakhpur')
 ]

_schema = "cust_id int, flight_id string , origin string , destination string"

df_flight = spark.createDataFrame(data = flights_data , schema= _schema)
df_flight.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/15 17:24:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-------+---------+---------+-----------+
|cust_id|flight_id|   origin|destination|
+-------+---------+---------+-----------+
|      1|  Flight1|    Delhi|  Hyderabad|
|      1|  Flight2|Hyderabad|      Kochi|
|      1|  Flight3|    Kochi|  Mangalore|
|      2|  Flight1|   Mumbai|    Ayodhya|
|      2|  Flight2|  Ayodhya|  Gorakhpur|
+-------+---------+---------+-----------+



In [2]:
from pyspark.sql.functions import col

df_final_stop = df_flight.alias("original_df_flight").join(df_flight.alias("new_df_flight"), 
               [col("original_df_flight.cust_id") == col("new_df_flight.cust_id"), \
                col("original_df_flight.destination") == col("new_df_flight.origin")], "left") \
                .select(col("original_df_flight.cust_id"),col("original_df_flight.destination"),
                        col("new_df_flight.destination").alias("next_stop")) \
                .where("next_stop is NULL")

In [3]:
df_origin = df_flight.alias("original_df_flight").join(df_flight.alias("new_df_flight"), 
               [col("original_df_flight.cust_id") == col("new_df_flight.cust_id"), \
                col("original_df_flight.origin") == col("new_df_flight.destination")], "left") \
                .select(col("original_df_flight.cust_id"),col("original_df_flight.origin"),
                        col("new_df_flight.origin").alias("previous_stop")) \
                .where("previous_stop is NULL")

In [4]:
df_final = df_final_stop.alias("destination") \
            .join(df_origin.alias("origin"), df_final_stop.cust_id == df_origin.cust_id, "inner") \
            .select(col("origin.cust_id"),col("origin.origin"),col("destination.destination")).show()

+-------+------+-----------+
|cust_id|origin|destination|
+-------+------+-----------+
|      1| Delhi|  Mangalore|
|      2|Mumbai|  Gorakhpur|
+-------+------+-----------+

